<a href="https://colab.research.google.com/github/chelynl/sentiment_analysis/blob/main/Sentiment_Analysis_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resources:<br>
https://github.com/hellotinah/youtube_sentiment_analysis/blob/main/comments_full_analysis.ipynb<br>
https://medium.com/analytics-vidhya/whatsapp-chat-analysis-using-python-88123e63c8ae<br>
https://regex101.com/

In [173]:
# import packages
import re
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from datetime import datetime

# !pip install emoji
# !pip install vaderSentiment
import emoji
from emoji import UNICODE_EMOJI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# !pip install TextBlob
from textblob import TextBlob
from textblob import Word

In [2]:
# Authenticate drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will start by uploading the text messages that have already been converted from a txt file to an excel file. However, it is still VERY messy with incorrect headers and formats. We will need to clean up the data as a dataframe in order to do analysis.

In [174]:
# Import original data
raw_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Between Messages/between_msgs.xlsx')

# See preview
raw_data.head()

,DATA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,"Saturday, April 8, 2017",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"PM 02:33, 💍 Wifey Bear ❤️ : Hiiii boooo",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"PM 02:33, 💍 Hubby Bear 🐻 : First",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"PM 02:39, 💍 Hubby Bear 🐻 : (milk_face)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"PM 02:40, 💍 Hubby Bear 🐻 : STICKER",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


All the useful content are under the `DATA` column and the rest are columns with NaN. These columns will be dropped.

In [179]:
# Remove unwanted cols
data1 = raw_data['DATA']

In [184]:
# Check number of na values
print('Nulls: ', sum(data1.isna()))

# Drop na values
data2 = data1.dropna()

# See preview
data2.head()

Nulls:  3153


0                    Saturday, April 8, 2017
1    PM 02:33, 💍 Wifey Bear ❤️ : Hiiii boooo
2          PM 02:33, 💍 Hubby Bear 🐻 : First 
3     PM 02:39, 💍 Hubby Bear 🐻 : (milk_face)
4         PM 02:40, 💍 Hubby Bear 🐻 : STICKER
Name: DATA, dtype: object

The data is large enough to drop null values without affecting the analysis. Now, we will remove ALL emojis showing up next to the sender's name and message body.

In [185]:
# Function for removing emoticons
def remove_emojis(text):
    regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    return regex_pattern.sub(r'',text)

In [211]:
# List to hold clean text without emojis
no_emojis_text = []

# Iterate each row and remove emojis and append to clean list
for row in data2:
  if isinstance(row, str)==True:
    clean_row = remove_emojis(row)
    no_emojis_text.append(clean_row)
  else:
    no_emojis_text.append(row)

# Add clean values as new col in a df
data3 = pd.DataFrame(no_emojis_text, columns=['data'])

# Preview df
data3.head()

,data
0,"Saturday, April 8, 2017"
1,"PM 02:33, Wifey Bear : Hiiii boooo"
2,"PM 02:33, Hubby Bear : First"
3,"PM 02:39, Hubby Bear : (milk_face)"
4,"PM 02:40, Hubby Bear : STICKER"


In [256]:
# See starting data dimension
data3.shape

(155289, 2)

Multiple information are under a single column, `data`. We need to split the `data` column into 4 separate columns including:
- Date Message Sent
- Time Message Sent
- Sender Name
- Message Body

***

We will use regular expressions to parse and organize the data.

In [237]:
""" Pattern that distinguishes date message sent:
    Saturday, April 8, 2017
"""

date_pattern = re.compile('\w+,\s\w+\s\d*,\s\d+')

# Store individual dates as new list
dates_list = []

# iterate over each date to check for date pattern
for row in data3['data']:
  if isinstance(row, str)==True:
    # Create date var and append to list if pattern exists
    if re.match(date_pattern, row):
      date = re.findall(date_pattern, row)[0]
      dates_list.append(date)
    # if the date pattern doesn't exist, the message is part of the previous date
    else:
      dates_list.append(date)
  else:
    dates_list.append(date)


# Preview dates list
dates_list[:5]

['Saturday, April 8, 2017',
 'Saturday, April 8, 2017',
 'Saturday, April 8, 2017',
 'Saturday, April 8, 2017',
 'Saturday, April 8, 2017']

In [229]:
# Add dates list as a separate column
data3['Date Message Sent'] = pd.Series(dates_list)

# Preview data
data3

,data,Date Message Sent
0,"Saturday, April 8, 2017","Saturday, April 8, 2017"
1,"PM 02:33, Wifey Bear : Hiiii boooo","Saturday, April 8, 2017"
2,"PM 02:33, Hubby Bear : First","Saturday, April 8, 2017"
3,"PM 02:39, Hubby Bear : (milk_face)","Saturday, April 8, 2017"
4,"PM 02:40, Hubby Bear : STICKER","Saturday, April 8, 2017"
...,...,...
155284,"PM 10:10, Wifey Bear : https://www.youtube.c...","Wednesday, December 30, 2020"
155285,"Thursday, December 31, 2020","Thursday, December 31, 2020"
155286,"AM 11:02, Wifey Bear : Good morning bear!","Thursday, December 31, 2020"
155287,"AM 11:02, Wifey Bear : Hope you have a fun t...","Thursday, December 31, 2020"


Now that we have `Date Message Sent` as a separate column, we can rid of the rows that contain duplicate information (thus, removing date info from the original `data` column).

In [235]:
# Drop dup dates
data4 = data3[data3['data']!=data3['Date Message Sent']]

# Reset index and drop old index
data5 = data4.reset_index().drop(columns=['index'])

# Preview data
data5

,data,Date Message Sent
0,"PM 02:33, Wifey Bear : Hiiii boooo","Saturday, April 8, 2017"
1,"PM 02:33, Hubby Bear : First","Saturday, April 8, 2017"
2,"PM 02:39, Hubby Bear : (milk_face)","Saturday, April 8, 2017"
3,"PM 02:40, Hubby Bear : STICKER","Saturday, April 8, 2017"
4,"PM 02:40, Hubby Bear : LOOOOOOL","Saturday, April 8, 2017"
...,...,...
154188,"PM 09:54, Wifey Bear : https://youtu.be/FOQg...","Wednesday, December 30, 2020"
154189,"PM 10:10, Wifey Bear : https://www.youtube.c...","Wednesday, December 30, 2020"
154190,"AM 11:02, Wifey Bear : Good morning bear!","Thursday, December 31, 2020"
154191,"AM 11:02, Wifey Bear : Hope you have a fun t...","Thursday, December 31, 2020"


We will repeat the same process to extract the time the message was sent and give it its own column.

In [242]:
time_pattern = '\w+\s\d+:\d+'

# Store individual times as new list
times_list = []

for row in data5['data']:
  if isinstance(row, str)==True:
    # Create time var and append to list if pattern exists
    if re.match(time_pattern, row):
      time = re.findall(time_pattern, row)[0]
      times_list.append(time)

# Preview times list
times_list[:5]

['PM 02:33', 'PM 02:33', 'PM 02:39', 'PM 02:40', 'PM 02:40']

In [244]:
# Add time as new col
data5['Time Message Sent'] = pd.Series(times_list)

# Drop nulls (there were some rows that did not begin with the time; this could be due to the row continuing a previous msg)
data6 = data5.dropna()

# Preview data
data6

,data,Date Message Sent,Time Message Sent
0,"PM 02:33, Wifey Bear : Hiiii boooo","Saturday, April 8, 2017",PM 02:33
1,"PM 02:33, Hubby Bear : First","Saturday, April 8, 2017",PM 02:33
2,"PM 02:39, Hubby Bear : (milk_face)","Saturday, April 8, 2017",PM 02:39
3,"PM 02:40, Hubby Bear : STICKER","Saturday, April 8, 2017",PM 02:40
4,"PM 02:40, Hubby Bear : LOOOOOOL","Saturday, April 8, 2017",PM 02:40
...,...,...,...
151437,"PM 03:04, Wifey Bear : Hiii bearrr","Thursday, September 10, 2020",PM 09:54
151438,"PM 03:04, Hubby Bear : you cant hide from me...","Thursday, September 10, 2020",PM 10:10
151439,"PM 03:04, Wifey Bear : :3","Thursday, September 10, 2020",AM 11:02
151440,"PM 03:04, Hubby Bear : cute bear <3","Thursday, September 10, 2020",AM 11:02


Extract the sender's name and give it its own column.

In [247]:
# Possible names of senders
senders = ['Hubby Bear', 'Wifey Bear']

# List to store sender names per message
sender_list = []

# iterate over each row in data
for row in data6['data']:
  # iterate over each possible sender
  for sender in senders:
    if isinstance(row, str)==True:
      # append to list if sender exists
      if sender in row:
        s = sender
        sender_list.append(s)

# Preview sender list
sender_list[:5]

['Wifey Bear', 'Hubby Bear', 'Hubby Bear', 'Hubby Bear', 'Hubby Bear']

In [249]:
# Add sender col
data6['Sender'] = pd.Series(sender_list)

# Drop null values
data7 = data6.dropna()

# Preview data
data7

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,data,Date Message Sent,Time Message Sent,Sender
0,"PM 02:33, Wifey Bear : Hiiii boooo","Saturday, April 8, 2017",PM 02:33,Wifey Bear
1,"PM 02:33, Hubby Bear : First","Saturday, April 8, 2017",PM 02:33,Hubby Bear
2,"PM 02:39, Hubby Bear : (milk_face)","Saturday, April 8, 2017",PM 02:39,Hubby Bear
3,"PM 02:40, Hubby Bear : STICKER","Saturday, April 8, 2017",PM 02:40,Hubby Bear
4,"PM 02:40, Hubby Bear : LOOOOOOL","Saturday, April 8, 2017",PM 02:40,Hubby Bear
...,...,...,...,...
148719,"PM 06:08, Wifey Bear : I’m really touched be...","Sunday, August 16, 2020",PM 03:06,Wifey Bear
148720,"PM 06:08, Wifey Bear : STICKER","Sunday, August 16, 2020",PM 03:06,Hubby Bear
148721,"PM 06:09, Wifey Bear : I also sent you some ...","Sunday, August 16, 2020",PM 03:09,Wifey Bear
148722,"PM 07:26, Hubby Bear : thank you bear! :)","Sunday, August 16, 2020",PM 03:09,Hubby Bear


Lastly, extract the message body from the `data` column and give it its own column.

In [250]:
# List to store message bodies
msg_list = []

# Keep track of errors
errors = 0

# Iterate over each row and separate out the message body from the rest of the content
for row in data7['data']:
  if isinstance(row, str)==True:
    try:
      # only select the message content and append it to the list
      msg = row.split(': ',1)[1]
      msg_list.append(msg)
    except:
      errors+=1

# See number of errors
print('errors:', errors)

# Preview message list
msg_list[:5]

errors: 2598


['Hiiii boooo', 'First ', '(milk_face)', 'STICKER', 'LOOOOOOL']

In [270]:
# Add message col
data7['Message'] = pd.Series(msg_list)

# Drop null values
data8 = data7.dropna()

# Drop the data col from the df
data9 = data8.drop(columns=['data'])
data9

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date Message Sent,Time Message Sent,Sender,Message
0,"Saturday, April 8, 2017",PM 02:33,Wifey Bear,Hiiii boooo
1,"Saturday, April 8, 2017",PM 02:33,Hubby Bear,First
2,"Saturday, April 8, 2017",PM 02:39,Hubby Bear,(milk_face)
3,"Saturday, April 8, 2017",PM 02:40,Hubby Bear,STICKER
4,"Saturday, April 8, 2017",PM 02:40,Hubby Bear,LOOOOOOL
...,...,...,...,...
146081,"Saturday, April 25, 2020",AM 06:57,Hubby Bear,I’m really touched bear
146082,"Saturday, April 25, 2020",AM 06:57,Hubby Bear,STICKER
146083,"Saturday, April 25, 2020",AM 06:58,Hubby Bear,I also sent you some resources on fb msger to ...
146084,"Saturday, April 25, 2020",AM 06:58,Hubby Bear,thank you bear! :)


We started with 155,289 observations (after removing duplicate dates) and ended up with 146,086 observations post-cleaning (retaining about 94% of the data, which is good). We have sufficient information to do analysis.

***

We will do further cleaning by converting dates and times to datetime objects and purifying the message bodies (removing stopwords, addressing spelling errors, etc.).

In [271]:
# Convert date and time columns to datetime objects
data9['Date Message Sent'] = pd.to_datetime(data9['Date Message Sent'], format='%A, %B %d, %Y')
data9['Time Message Sent'] = pd.to_datetime(data9['Time Message Sent'], format='%p %I:%M').dt.time